In [2]:
from sqlalchemy import create_engine 
import pandas as pd
import tushare as ts
from openpyxl import load_workbook
from WindPy import w
from scipy.spatial.distance import pdist, euclidean
import itertools
from pathlib import Path
# 取数据
# w.start()

In [ ]:
w.wsd("000045.OF", "prt_topstockcode,prt_topsecuritiecode,prt_topsecuritievalue,prt_heavilyheldsectoasset", "2022-11-30", "2023-03-02", "order=1;topNum=1;unit=1;Period=Q")

# 分析RMSE

In [5]:


df_new_info = pd.read_excel('C:/Users/youth/Desktop/中泰金工/固收+基金仓位探测/detect_fund_positions20230303/data/固收+基金名单(回溯12个季度)_to_2022-12-31.xlsx')
fund_code_mix1 = df_new_info[df_new_info['投资类型(二级)'].isin(['混合债券型一级基金'])]['基金代码'].to_list()
fund_code_mix2 = df_new_info[df_new_info['投资类型(二级)'].isin(['混合债券型二级基金'])]['基金代码'].to_list()
fund_code_flexible = df_new_info[df_new_info['投资类型(二级)'].isin(['灵活配置型基金'])]['基金代码'].to_list()
fund_code_biasb = df_new_info[df_new_info['投资类型(二级)'].isin(['偏债混合型基金'])]['基金代码'].to_list()
fund_class = df_new_info['投资类型(二级)'].to_list()
fund_list = df_new_info['基金代码'].to_list()
info = list(zip(fund_list, fund_class))
info

[('000045.OF', '混合债券型二级基金'),
 ('000047.OF', '混合债券型一级基金'),
 ('000054.OF', '混合债券型二级基金'),
 ('000065.OF', '灵活配置型基金'),
 ('000072.OF', '灵活配置型基金'),
 ('000107.OF', '混合债券型二级基金'),
 ('000110.OF', '偏债混合型基金'),
 ('000118.OF', '混合债券型二级基金'),
 ('000121.OF', '偏债混合型基金'),
 ('000130.OF', '混合债券型一级基金'),
 ('000142.OF', '混合债券型二级基金'),
 ('000143.OF', '混合债券型二级基金'),
 ('000149.OF', '混合债券型一级基金'),
 ('000171.OF', '混合债券型二级基金'),
 ('000181.OF', '混合债券型二级基金'),
 ('000189.OF', '混合债券型二级基金'),
 ('000190.OF', '灵活配置型基金'),
 ('000195.OF', '灵活配置型基金'),
 ('000207.OF', '混合债券型一级基金'),
 ('000215.OF', '灵活配置型基金'),
 ('000227.OF', '混合债券型一级基金'),
 ('000236.OF', '混合债券型二级基金'),
 ('000239.OF', '混合债券型一级基金'),
 ('000244.OF', '混合债券型一级基金'),
 ('000254.OF', '混合债券型一级基金'),
 ('000310.OF', '混合债券型一级基金'),
 ('000314.OF', '灵活配置型基金'),
 ('000338.OF', '混合债券型二级基金'),
 ('000367.OF', '偏债混合型基金'),
 ('000377.OF', '混合债券型二级基金'),
 ('000385.OF', '混合债券型二级基金'),
 ('000406.OF', '混合债券型二级基金'),
 ('000436.OF', '偏债混合型基金'),
 ('000463.OF', '混合债券型二级基金'),
 ('000507.OF', '偏债混合型基金'),
 ('0005

In [27]:
df_RMSE = pd.read_csv('所有基金探测仓位的平均标准差.csv').set_index('code')
mix1Rseult = df_RMSE.loc[fund_code_mix1]
mix2Rseult = df_RMSE.loc[fund_code_mix2]
flexibleRseult = df_RMSE.loc[fund_code_flexible]
biasbRseult = df_RMSE.loc[fund_code_biasb]
mix1Rseult.loc['mean'] = mix1Rseult.mean(axis=0)
mix2Rseult.loc['mean'] = mix2Rseult.mean(axis=0)
flexibleRseult.loc['mean'] = flexibleRseult.mean(axis=0)
biasbRseult.loc['mean'] = biasbRseult.mean(axis=0)
mix1_RMSE = pd.DataFrame(mix1Rseult.loc['mean']).T
mix2_RMSE = pd.DataFrame(mix2Rseult.loc['mean']).T
flexible_RMSE = pd.DataFrame(flexibleRseult.loc['mean']).T
biasb_RMSE = pd.DataFrame(biasbRseult.loc['mean']).T
df_everyRMSE = pd.concat([mix1_RMSE, mix2_RMSE, flexible_RMSE, biasb_RMSE], axis=0)
df_everyRMSE.to_excel('各类型固收+平均RMSE.xlsx')

In [25]:
mix1_RMSE

,R1,R2,R3,R4,R5,R6,R7,R8,R9
mean,7.531196,12.248804,4.145109,7.53087,24.347826,24.323696,24.349239,4.634239,7.537609


# 分析仓位波动

In [29]:
df_std = pd.read_csv('所有基金探测仓位的平均标准差.csv').set_index('code')
mix1Rseult = df_std.loc[fund_code_mix1]
mix2Rseult = df_std.loc[fund_code_mix2]
flexibleRseult = df_std.loc[fund_code_flexible]
biasbRseult = df_std.loc[fund_code_biasb]
mix1Rseult.loc['mean'] = mix1Rseult.mean(axis=0)
mix2Rseult.loc['mean'] = mix2Rseult.mean(axis=0)
flexibleRseult.loc['mean'] = flexibleRseult.mean(axis=0)
biasbRseult.loc['mean'] = biasbRseult.mean(axis=0)
mix1_std = pd.DataFrame(mix1Rseult.loc['mean']).T
mix2_std = pd.DataFrame(mix2Rseult.loc['mean']).T
flexible_std = pd.DataFrame(flexibleRseult.loc['mean']).T
biasb_std = pd.DataFrame(biasbRseult.loc['mean']).T
df_everystd = pd.concat([mix1_std, mix2_std, flexible_std, biasb_std], axis=0)
df_everystd.to_excel('各类型固收+平均仓位波动.xlsx')

df_everystd

,std1,std3,std4,std8
mean,9.175698,8.738565,9.178807,8.936533
mean,8.417778,8.042665,8.418362,8.206413
mean,7.221697,7.142569,7.221434,7.237726
mean,8.479748,8.088350,8.481328,8.328016


# 将所有RMSE结果按照固收+类别分类

In [13]:
df_allRMSE = pd.read_csv('c=100, 半衰期加权,所有基金探测仓位的平均RMSE.csv').set_index('code')
df_allRMSE.columns = ['WLScons_amendQP_wreg', 'WLSmaxmin_cons_wreg', 'WLSupdown_cons_wreg', 'OLScons_amendQP_wreg', 'WLScons_linear_reg', 'WLSridge_reg', 'WLSlasso_reg', 'updown_cons_dreg', 'cons_amendQP_dreg']

mix1Rseult = df_allRMSE.loc[fund_code_mix1]
mix2Rseult = df_allRMSE.loc[fund_code_mix2]
flexibleRseult = df_allRMSE.loc[fund_code_flexible]
biasbRseult = df_allRMSE.loc[fund_code_biasb]
with pd.ExcelWriter('研报需要使用的其他数据整理统计.xlsx', engine='openpyxl', mode='a') as xlsx:
    mix1Rseult.to_excel(xlsx, '混合债券型级基金')
    mix2Rseult.to_excel(xlsx, '混合债券型二基金')
    flexibleRseult.to_excel(xlsx, '灵活配型基金')
    biasbRseult.to_excel(xlsx, '偏债合型基金')
    # prt2.to_excel(xlsx, 'WLSmaxmin_cons_wreg')
    # prt5.to_excel(xlsx, 'WLScons_linear_reg')
    # prt6.to_excel(xlsx, 'WLSridge_reg')
    # prt7.to_excel(xlsx, 'WLSlasso_reg')
    # prt9.to_excel(xlsx, 'cons_amendQP_dreg')